# Figure 10: Glacier signature uncertainty

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd

import os
from glob import glob

import plotly.express as px
import plotly.graph_objects as go
from   plotly.subplots import make_subplots

cl = px.colors.qualitative.D3
os.chdir('/home/rooda/Dropbox/Patagonia/')

## Data

In [ ]:
# feature importance data
data = pd.read_csv("MS2 Results/feature_importance_rmse.csv", index_col = "ID")
area = pd.read_csv("MS2 Results/zenodo/dataset_historical.csv", index_col = "basin_id").area_RGI6
data = data.join(area)

In [ ]:
# weighted-means
metrics   = data.Metric.unique()
variables = data.Variable.unique()
sources   = data.columns[2:8]

df_wm = []

for variable in variables: 
    for metric in metrics: 
        data_i = data[(data.Variable == variable) & (data.Metric == metric)]
        
        for source in sources: 
            data_s = (data_i[source] * data_i.area_RGI6).sum() / data_i.area_RGI6.sum()
            df_wm.append([variable, metric, source, data_s])
            
df_wm = pd.DataFrame(df_wm, columns = ["Variable", "Metric", "Source", "WM"])

## Plot

In [ ]:

metrics = metrics.reshape(2,5)
titles = ['Reference magnitude', 'Peak water year', 'Peak water magnitude', 'Peak water duration', 'Inter-annual variability', 
          'Reference<br>seasonal contribution', 'Reference<br>seasonal variability', 'Seasonal shift', 'Long-term trend', 'Long-term change']

cl = px.colors.colorbrewer.Dark2

fig = make_subplots(rows=2, cols=5, horizontal_spacing = 0.01, vertical_spacing = 0.08, shared_yaxes = True, 
                    subplot_titles = titles, x_title='Feature importance (% of RMSE loss after permutations)')

# a to e) Boxplot -----------------------------------------------------------------------------------------------------------
for y in range (0,2):
    for x in range(0,5):   
        
        # melt on glacier
        data_xy = data[(data.Metric == metrics[y,x]) & (data.Variable == "melt_on_glacier")]
        data_xy_wm = df_wm[(df_wm.Metric == metrics[y,x]) & (df_wm.Variable == "melt_on_glacier")].set_index("Source")
        
        fig.add_trace(go.Box(x = data_xy.Outline, name = "Glacier<br>inventory", marker_color=cl[0], marker = dict(size= 3), opacity=0.4, offsetgroup= "A", showlegend = False, boxmean=False), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = data_xy.Volume,  name = "Glacier<br>volume",    marker_color=cl[4], marker = dict(size= 3), opacity=0.4, offsetgroup= "A", showlegend = False, boxmean=False), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = data_xy.Climate, name = "Reference<br>climate", marker_color=cl[1], marker = dict(size= 3), opacity=0.4, offsetgroup= "A", showlegend = False, boxmean=False), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = data_xy.GCM,     name = "GCM",                  marker_color=cl[5], marker = dict(size= 3), opacity=0.4, offsetgroup= "A", showlegend = False, boxmean=False), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = data_xy.SSP,     name = "SSP",                  marker_color=cl[6], marker = dict(size= 3), opacity=0.4, offsetgroup= "A", showlegend = False, boxmean=False), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = data_xy.BCM,     name = "Bias<br>correction",   marker_color=cl[2], marker = dict(size= 3), opacity=0.4, offsetgroup= "A", showlegend = False, boxmean=False), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = np.array(data_xy_wm.WM.Outline), name = "Glacier<br>inventory", marker_color=cl[0], boxpoints="all", line_color= "rgba(255,255,255,0)", opacity=0.5, pointpos = -0.15, marker = dict(size= 8), offsetgroup= "A", showlegend = False), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = np.array(data_xy_wm.WM.Volume),  name = "Glacier<br>volume",    marker_color=cl[4], boxpoints="all", line_color= "rgba(255,255,255,0)", opacity=0.5, pointpos = -0.15, marker = dict(size= 8), offsetgroup= "A", showlegend = False), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = np.array(data_xy_wm.WM.Climate), name = "Reference<br>climate", marker_color=cl[1], boxpoints="all", line_color= "rgba(255,255,255,0)", opacity=0.5, pointpos = -0.15, marker = dict(size= 8), offsetgroup= "A", showlegend = False), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = np.array(data_xy_wm.WM.GCM),     name = "GCM",                  marker_color=cl[5], boxpoints="all", line_color= "rgba(255,255,255,0)", opacity=0.5, pointpos = -0.15, marker = dict(size= 8), offsetgroup= "A", showlegend = False), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = np.array(data_xy_wm.WM.SSP),     name = "SSP",                  marker_color=cl[6], boxpoints="all", line_color= "rgba(255,255,255,0)", opacity=0.5, pointpos = -0.15, marker = dict(size= 8), offsetgroup= "A", showlegend = False), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = np.array(data_xy_wm.WM.BCM),     name = "Bias<br>correction",   marker_color=cl[2], boxpoints="all", line_color= "rgba(255,255,255,0)", opacity=0.5, pointpos = -0.15, marker = dict(size= 8), offsetgroup= "A", showlegend = False), row=y+1, col=x+1)
 
        # total runoff
        data_xy = data[(data.Metric == metrics[y,x]) & (data.Variable == "total_runoff")]
        data_xy_wm = df_wm[(df_wm.Metric == metrics[y,x]) & (df_wm.Variable == "total_runoff")].set_index("Source")
        
        fig.add_trace(go.Box(x = data_xy.Outline, name = "Glacier<br>inventory", marker_color=cl[0], marker = dict(size= 3), opacity=0.8, offsetgroup= "B", showlegend = False, boxmean=False), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = data_xy.Volume,  name = "Glacier<br>volume",    marker_color=cl[4], marker = dict(size= 3), opacity=0.8, offsetgroup= "B", showlegend = False, boxmean=False), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = data_xy.Climate, name = "Reference<br>climate", marker_color=cl[1], marker = dict(size= 3), opacity=0.8, offsetgroup= "B", showlegend = False, boxmean=False), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = data_xy.GCM,     name = "GCM",                  marker_color=cl[5], marker = dict(size= 3), opacity=0.8, offsetgroup= "B", showlegend = False, boxmean=False), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = data_xy.SSP,     name = "SSP",                  marker_color=cl[6], marker = dict(size= 3), opacity=0.8, offsetgroup= "B", showlegend = False, boxmean=False), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = data_xy.BCM,     name = "Bias<br>correction",   marker_color=cl[2], marker = dict(size= 3), opacity=0.8, offsetgroup= "B", showlegend = False, boxmean=False), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = np.array(data_xy_wm.WM.Outline), name = "Glacier<br>inventory", marker_color=cl[0], line_color= "rgba(255,255,255,0)", boxpoints="all",  opacity=0.9, pointpos = -0.15, marker = dict(size= 8), offsetgroup= "B", showlegend = False), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = np.array(data_xy_wm.WM.Volume),  name = "Glacier<br>volume",    marker_color=cl[4], line_color= "rgba(255,255,255,0)", boxpoints="all", opacity=0.9, pointpos = -0.15, marker = dict(size= 8), offsetgroup= "B", showlegend = False), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = np.array(data_xy_wm.WM.Climate), name = "Reference<br>climate", marker_color=cl[1], line_color= "rgba(255,255,255,0)", boxpoints="all", opacity=0.9, pointpos = -0.15, marker = dict(size= 8), offsetgroup= "B", showlegend = False), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = np.array(data_xy_wm.WM.GCM),     name = "GCM",                  marker_color=cl[5], line_color= "rgba(255,255,255,0)", boxpoints="all", opacity=0.9, pointpos = -0.15, marker = dict(size= 8), offsetgroup= "B", showlegend = False), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = np.array(data_xy_wm.WM.SSP),     name = "SSP",                  marker_color=cl[6], line_color= "rgba(255,255,255,0)", boxpoints="all", opacity=0.9, pointpos = -0.15, marker = dict(size= 8), offsetgroup= "B", showlegend = False), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = np.array(data_xy_wm.WM.BCM),     name = "Bias<br>correction",   marker_color=cl[2], line_color= "rgba(255,255,255,0)", boxpoints="all", opacity=0.9, pointpos = -0.15, marker = dict(size= 8), offsetgroup= "B", showlegend = False), row=y+1, col=x+1)
 
        
fig.update_annotations(font_size=14)
fig.update_traces(line_width = 1.5)
fig.add_annotation(text="Glacier runoff", font=dict(size=12),  arrowhead=0, arrowwidth=1, arrowcolor="rgb(30,30,30)", ax=40, x=0.05, y=4.2, showarrow=True, row=1, col=1)
fig.add_annotation(text="Glacier melt", font=dict(size=12),  arrowhead=0, arrowwidth=1, arrowcolor="rgb(30,30,30)", ax=70, x=0.05, y=3.8, showarrow=True, row=1, col=1)

fig.update_xaxes(range = [-0.12, 0.85], row = 1, col = 2)
fig.update_xaxes(range = [-0.05, 1.1], row = 2, col = 2)
fig.update_xaxes(range = [-0.1, 0.6], row = 2, col = 3)
fig.update_xaxes(range = [-0.20, 0.8], row = 2, col = 4)
fig.update_xaxes(range = [-0.05, 0.7], row = 2, col = 5)


# general 
fig.update_layout(boxmode='group', boxgap=0.15)
fig.update_layout(width = 900, height =900, template = "seaborn", autosize = False, margin = dict(l=10, r=10, b=50, t=30, pad=0, autoexpand=True))
fig.update_xaxes(griddash = "dot", gridcolor = "rgba(255,255,255,0.8)", zeroline=False, showline = True, linecolor = 'black', linewidth = 1, tickformat = ',.0%', dtick = 0.3, ticks="outside", mirror=True)
fig.update_yaxes(griddash = "dot", gridcolor = "rgba(255,255,255,0.8)", zeroline=True,  showline = True, linecolor = 'black', linewidth = 1, mirror=True)
fig.show()

fig.write_image("/home/rooda/Dropbox/Patagonia/MS2 Results/Figure_10_signature.png", scale=4)

In [ ]:
"This was especially clear for {} where the reference climate accumulated more than 50% of the total RMSE loss after the permutations".format(
df_wm[(df_wm.Source == "Climate") & (df_wm.WM > 0.5)  & (df_wm.Variable ==  "melt_on_glacier")].Metric.values)

In [ ]:
df_wm[(df_wm.Variable ==  "melt_on_glacier") & df_wm.Source.isin(["Climate", "Volume", "Outline"])].groupby(by = ["Metric"]).sum(numeric_only = True) > df_wm[
      (df_wm.Variable ==  "melt_on_glacier") & df_wm.Source.isin(["SSP",     "GCM",    "BCM"])].groupby(by = ["Metric"]).sum(numeric_only = True)

In [ ]:
df_wm[(df_wm.Variable ==  "total_runoff") & df_wm.Source.isin(["Climate", "Volume", "Outline"])].groupby(by = ["Metric"]).sum(numeric_only = True) > df_wm[
      (df_wm.Variable ==  "total_runoff") & df_wm.Source.isin(["SSP",     "GCM",    "BCM"])].groupby(by = ["Metric"]).sum(numeric_only = True)